In [1]:
# モジュールのインポート
import os
import shutil
from pathlib import Path
from uuid import uuid4
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate

In [2]:
# chromaデータベースを初期化
CHROMA_DIR = "./chroma_langchain_db"
if os.path.exists(CHROMA_DIR):
    shutil.rmtree(CHROMA_DIR)

In [3]:
# LLM、埋め込みモデルを初期化
llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

C:\Users\1109685\AppData\Local\Temp\ipykernel_7676\3608532029.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [4]:
# pdfの読み込み
pdf_path="sample.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} pages from {pdf_path}")

# ドキュメントを分割（オプション：チャンクサイズの調整）
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(documents)
print(f"Split into {len(split_docs)} chunks")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)


Loaded 3 pages from sample.pdf
Split into 3 chunks


In [5]:
# ドキュメントをベクトルストアに追加
file_paths = [pdf_path] * len(split_docs)  # 各ドキュメントのファイルパスを保持

# Documentオブジェクトを作成
docs = [
    Document(page_content=doc.page_content, metadata={"file_path": file_path})
    for doc, file_path in zip(split_docs, file_paths)
]

# 各ドキュメントにユニークなUUIDを生成
uuids = [str(uuid4()) for _ in range(len(docs))]

# ベクトルストアにドキュメントを追加
db.add_documents(documents=docs, ids=uuids)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [4]:
# PDFをデータベースに保存
def pdf_loader(pdf_path: str, persist_directory: str = "./chroma_langchain_db") -> Chroma:
    global db

    # chromaデータベースを初期化
    CHROMA_DIR = "./chroma_langchain_db"
    if os.path.exists(CHROMA_DIR):
        shutil.rmtree(CHROMA_DIR)
        
    # # 既存のデータベースを削除して初期化
    # if os.path.exists(persist_directory):
    #     shutil.rmtree(persist_directory)
    #     print("ベクトルストアを初期化しました")
    
    # ベクトルストアを構築
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    db = Chroma(
        collection_name="example_collection",
        embedding_function=embeddings,
        persist_directory=persist_directory,
    )
    
    # PDF読み込み
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from {pdf_path}")
    
    # 分割
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = splitter.split_documents(documents)
    print(f"Split into {len(split_docs)} chunks")
    
    # 元のメタデータを保持しつつ、ファイルパスを追加
    docs = []
    for doc in split_docs:
        # 元のメタデータをコピー
        metadata = doc.metadata.copy() if hasattr(doc, 'metadata') and doc.metadata else {}
        # ファイルパスを追加
        metadata["file_path"] = pdf_path
        docs.append(Document(page_content=doc.page_content, metadata=metadata))
    
    # UUID生成
    uuids = [str(uuid4()) for _ in range(len(docs))]
    
    # ベクトルストアに追加
    db.add_documents(documents=docs, ids=uuids)
    # db.persist()  # データベースを永続化
    print(f"{pdf_path}が保存されました")
    
    return db  # 作成したdbインスタンスを返す

In [5]:
# RAGの実装
def answer_RAG(query: str, top_k: int = 2) -> str:
    global db
    search_results=db.similarity_search(query, k=top_k)

    # 検索結果からドキュメントをまとめる
    context = "\n".join([result.page_content for result in search_results])

    # LLMに質問を投げる
    prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはチャットボットの回答作成者です。"
                ),
                (
                    "human",
                    "次の文脈に基づいて質問に回答してください。\n\n"
                    "質問: {context}\n\n"
                    "質問：{query}\n"
                )
            ]
    )
    chain = prompt | llm

    answer = chain.invoke({"context": context, "query": query})
    return answer

In [ ]:
# queryを設定して実行
pdf_path="sample.pdf"
query="セクション2の内容は"

pdf_loader(pdf_path)
answer=answer_RAG(query)
print(answer.content)

PermissionError: [WinError 32] プロセスはファイルにアクセスできません。別のプロセスが使用中です。: './chroma_langchain_db\\chroma.sqlite3'

: 